# Profiling

## Available profilers in Python

### lightweight
 -  **%timeit** A very usefull magic function (especially for this course!)
 -  **time** (module) This module provides various time-related functions.
 
### Standard
 -  **cProfile** (module) This module is recommended for most users; it’s a C extension with reasonable overhead that makes it suitable for profiling long-running programs. Based on lsprof, contributed by Brett Rosen and Ted Czotter. Default CPU profiler, a bit slow (deterministic)
 - **pyinstrument** Reports the call stack and elapsed times (statistical)
 - **yappi** Allows to profile multi-threaded applications (deterministic)
 - **memory_profiler** Monitors memory consumption of a process
 - **line_profiler** Profile the time individual lines of code take to execute

### Collection of Profilers
- **decoProf** Is a python tool, that bundles the above profilers and allows user's to use select the profiler they want to profile the funcion they want. https://github.com/SURFQuantum/decoProf

#### "deterministic" and "statistical" profilers:
> **_NOTE:_**  This description is taken from the decoProf documentaion https://github.com/SURFQuantum/decoProf

**Deterministic**

Deterministic profilers work by hooking into several function call/leave events and calculate all metrics according to these.

**Statistical**

Statistical profilers do not track every function call the program makes but they record the call stack every 1ms or whatever defined in the interval. The statistical profilers can impose less overhead compared to the deterministic ones.

In [2]:
import cProfile

cProfile.run('explicit_matmul(A,B,C)') #By default the run method prints to the std out


         160808 function calls in 6.684 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    40201    0.041    0.000    0.094    0.000 <__array_function__ internals>:177(shape)
        1    6.590    6.590    6.684    6.684 <ipython-input-1-7ccde38beb56>:7(explicit_matmul)
        1    0.000    0.000    6.684    6.684 <string>:1(<module>)
    40201    0.009    0.000    0.009    0.000 fromnumeric.py:1987(_shape_dispatcher)
    40201    0.015    0.000    0.015    0.000 fromnumeric.py:1991(shape)
        1    0.000    0.000    6.684    6.684 {built-in method builtins.exec}
    40201    0.030    0.000    0.044    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




## Memory profiler
> https://github.com/pythonprofilers/memory_profiler

In [4]:
pip install -U memory_profiler

mprof: Sampling memory every 0.1s
running new process
running as a Python program...
Serial:  6.472360283
Using last profile data.


In [ ]:
%%writefile mem_growth.py
class simple_class:
    """A simple example class"""
    i = 100000000000000000.0

    def f(self):
        return 'hello world'
    
@profile
def dumb():
    """This function will just keep allocating a class"""
    y = []
    for i in range(5000000):
        x = simple_class()
        y.append(x.i + float(i))
    return(y)

@profile
def not_as_dumb():
    """This function will just keep allocating a class, and maybe try to delete it...."""
    y = []
    x = simple_class()
    for i in range(5000000):
        y.append(x.i + float(i))
        
    return(y)

        
if __name__ == "__main__":
    dumb()
    not_as_dumb()        


In [37]:
!mprof run mem_growth.py
!mprof plot --output=memory.png

In [ ]:
cProfile.run('explicit_matmul(A,B)',"my_perf_file.out") #By default the run method prints to the std out

In [ ]:
import pstats
from pstats import SortKey

p = pstats.Stats('my_perf_file.out')  #read in the profile data

#you can sort by the internal time
p.sort_stats('time')
p.print_stats()

#you can sort by the number of calls
p.sort_stats('calls')
p.print_stats()

#you can reverse the order
p.reverse_order()
p.print_stats()


In [ ]:
import cProfile

def do_profile(func):
    def profiled_func(*args, **kwargs):
        profile = cProfile.Profile()
        try:
            profile.enable()
            result = func(*args, **kwargs)
            profile.disable()
            return result
        finally:
            profile.print_stats()
    return profiled_func

In [ ]:
# Simple Matrix multiplication algorithm
@do_profile
def numpy_matmul(A,B):
    npA = np.array(A)
    npB = np.array(B)
    C = np.matmul(A,B)
    return C

@do_profile
def explicit_matmul(A,B):
    C = [[0 for x in range(len(A))] for y in range(len(B[0]))]
    for i in range(len(A)):
        for j in range(len(B[0])):
            for k in range(len(B)):
                C[i][j] += A[i][k] * B[k][j]
    return C

#Set matrix dimension
AX=AY=BX=BY=100

#Define Matrix A
A = [[random() for x in range(AX)] for y in range(AY)]

#Define Matrix B
B = [[random() for x in range(BX)] for y in range(BY)]

res = numpy_matmul(A,B)

res = explicit_matmul(A,B)